In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import csv
import json

## Authentication

In [2]:
CLIENT_ID = 'd8415d5149674a91a7e81de017c789d7'
CLIENT_SECRET = '576b0598e8014aeb93f7bc1ab69ac410'

client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Playlists

In [3]:
PLAYLIST_ID_MUSICAL_EXPERIMENT = 'https://open.spotify.com/playlist/3tTo8cKpIQbLJ3MTzgEVjV?si=5729dd4f3bcc41fe'

PLAYLIST_ID_MUSICAL_EXPERIMENT_2 = 'https://open.spotify.com/playlist/1Kzd8jonR1022i8dL8eDda?si=615bed3d56b64ed7'

### Mainstream Genres

In [4]:
MAINSTREAM_GENRES_FILE = 'datasets/mainstream_genres.csv'

mainstream_genres = list()

with open(MAINSTREAM_GENRES_FILE, 'r') as f:
    csvreader = csv.reader(f, delimiter='\n')
    for row in csvreader:
        mainstream_genres.append(row[0])
    
    print(mainstream_genres)

['pop', 'hip hop', 'hip pop', 'rap', 'trap', 'drill', 'rock', 'blues', 'jazz', 'country', 'folk', 'r&b', 'motown', 'soul', 'classical', 'electronica', 'electro', 'electropop', 'disco', 'dance', 'invasion', 'emo', 'punk', 'post-grunge', 'grunge', 'metal', 'metalcore', 'reggae', 'k-pop', 'urbano', 'reggaeton', 'latin', 'sertanejo', 'gospel', 'mpb', 'lo-fi', 'soundtrack', 'worship']


## Get Artist Data

In [5]:
def get_artist_genres(artist_id:str) -> list:
    artist = spotify.artist(artist_id)

    return artist['genres']


def get_related_artists_names(artist_id:str) -> list:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_names = [related_artist['name'] for related_artist in related_artists['artists']]

    return related_artists_names

def get_related_artists_genres(artist_id:str, track_genres:list) -> set:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_genres = set()

    for related_artist in related_artists['artists']:
        genres = set(related_artist['genres'])

        for genre in genres:
            related_artists_genres.add(genre)

        print(related_artists_genres)

    return list(related_artists_genres - set(track_genres))

def get_generic_genres(mainstream_genres:list, track_genres:list, track_related_genres:list) -> list:
    generic_genres = list()

    for track_genre in track_genres:
        for mainstream_genre in mainstream_genres:
            if mainstream_genre in track_genre:
                generic_genres.append(mainstream_genre)

    for track_related_genre in track_related_genres:
        for mainstream_genre in mainstream_genres:
            if mainstream_genre in track_related_genre:
                generic_genres.append(mainstream_genre)

    return generic_genres

## Get Track Data

In [6]:
def get_tracks_id(playlist_id:str) -> list:  
    tracks_id = list()
    playlist_items = spotify.playlist_items(playlist_id)

    for item in playlist_items['items']:
        track = item['track']
        tracks_id.append(track['id'])

    return tracks_id

def get_track_data(track_id:str) -> dict:
    metadata = spotify.track(track_id)
    audio_features = spotify.audio_features(track_id)

    artist_id = metadata['album']['artists'][0]['id']
    track_genres = get_artist_genres(artist_id)
    related_track_genres = get_related_artists_genres(artist_id, track_genres)


    return {
        'id': metadata['id'],
        'name': metadata['name'],
        'album': metadata['album']['name'],
        'artist': metadata['album']['artists'][0]['name'],
        'genres': track_genres,
        'related_artists': get_related_artists_names(artist_id),
        'related_genres': related_track_genres,
        'generic_genres': get_generic_genres(mainstream_genres,track_genres, related_track_genres),
        'release_date': metadata['album']['release_date'],
        'duration_ms': metadata['duration_ms'],
        'popularity': metadata['popularity'],
        'acousticness': audio_features[0]['acousticness'],
        'danceability': audio_features[0]['danceability'],

        'energy': audio_features[0]['energy'],
        'instrumentalness': audio_features[0]['instrumentalness'],
        'key': audio_features[0]['key'],
        'liveness': audio_features[0]['liveness'],
        'loudness': audio_features[0]['loudness'],
        'mode': audio_features[0]['mode'],
        'speechiness': audio_features[0]['speechiness'],
        'tempo': audio_features[0]['tempo'],
        'time_signature': audio_features[0]['time_signature'],
        'valence': audio_features[0]['valence'],
    }

In [7]:
tracks_id = get_tracks_id(PLAYLIST_ID_MUSICAL_EXPERIMENT_2)

In [8]:
tracks_data = list()

for track_id in tracks_id:
    time.sleep(.5)
    tracks_data.append(get_track_data(track_id))
    
print(tracks_data)


{'psychedelic rock', 'rock', 'album rock', 'singer-songwriter', 'supergroup', 'hard rock', 'folk rock', 'british blues', 'blues rock', 'classic rock'}
{'blues', 'psychedelic rock', 'rock', 'electric blues', 'album rock', 'singer-songwriter', 'supergroup', 'hard rock', 'folk rock', 'british blues', 'country rock', 'blues rock', 'classic rock'}
{'blues', 'psychedelic rock', 'rock', 'electric blues', 'album rock', 'singer-songwriter', 'supergroup', 'hard rock', 'folk rock', 'british blues', 'country rock', 'blues rock', 'classic rock', 'british invasion'}
{'mellow gold', 'blues', 'psychedelic rock', 'rock', 'electric blues', 'album rock', 'singer-songwriter', 'supergroup', 'hard rock', 'folk rock', 'british blues', 'country rock', 'blues rock', 'classic rock', 'heartland rock', 'british invasion'}
{'mellow gold', 'soft rock', 'rock', 'singer-songwriter', 'blues rock', 'british invasion', 'psychedelic rock', 'electric blues', 'folk rock', 'classic rock', 'heartland rock', 'proto-metal', 'b

In [9]:
with open("playlist.json", "w") as f:
    json.dump(tracks_data, f, indent=4)